In [1]:
import sys
sys.path.append(sys.path[0]+'/../..')

import utils.climatology

In [2]:
import xarray
import numpy
import pandas

# Make up some test data

In [3]:
 # Create an xarray of cftime.datetime objects
testTime = xarray.cftime_range(start="1200-01-15", periods=1000, freq="M", calendar="noleap")
startYear=1208
finishYear=1225

In [4]:
testData = 1000+10*numpy.random.randn(1000,1)

testDa = xarray.DataArray(
    testData, 
    dims=('time', 'index'), 
    coords = {'time':testTime}
)

# Test date interval

In [5]:
expClimat=testData[(startYear-1200)*12:(finishYear-1200+1)*12].copy()

In [6]:
#This is a efficient but unconvincing (due to obfuscation) way of testing it

if numpy.alltrue(
    numpy.equal(expClimat, 
                utils.climatology.dateInterval(testDa, startYear, finishYear).values
               )
):
    print('Success, utils.climatology.dateInterval works')   
else:
    raise RuntimeError('Error found. testClimatology of dateInterval failed')
    

Success, utils.climatology.dateInterval works


In [7]:
# the harder way to stuff up of testing that

calculatedClimat=utils.climatology.dateInterval(testDa, startYear, finishYear)

for i in range(0,len(expClimat)):
    if calculatedClimat[i]!=expClimat[i]:
        raise RuntimeError('Error found. testClimatology of dateInterval failed')
        

# Test normalise

In [8]:
# For my climatology, calculate mean and std dev

expMean = list()
expStd = list()
expNormalised = list()

for iMonth in range(0,12):
    monthsC=numpy.arange(iMonth, 12*(finishYear-startYear+1), 12)
    monthClimat=(expClimat[monthsC])
    expMean=(numpy.mean(monthClimat))
    expStd=(numpy.std(monthClimat))
    monthsT=numpy.arange(iMonth, len(testData), 12)
    expNormalised.append([(testData[monthsT]-expMean)/expStd])


In [9]:
calculatedNormalised=utils.climatology.normalise(testDa, calculatedClimat)

In [10]:
for iMonth in range(0,12):
    calcNorm=calculatedNormalised.where(calculatedNormalised.month==iMonth+1, drop=True).values
    for i in range(0,len(expNormalised)):
        if not numpy.isclose(calcNorm[i],expNormalised[iMonth][0][i]):
            raise RuntimeError('Error found. testClimatology of dateInterval failed')
    

In [11]:
#check this doesn't have a month index (aka same number of dimensions we started with)

calculatedNormalised

<xarray.DataArray (time: 1000, index: 1)>
array([[ 1.63367602],
       [-1.52572663],
       [-1.71779092],
       [ 0.51994478],
       [ 1.2154639 ],
       [ 1.85694266],
       [-0.02789604],
       [-0.79745415],
       [-0.45843986],
       [ 1.22096307],
       [-0.06405327],
       [ 0.80998486],
       [-0.88965918],
       [-0.84866238],
       [-1.43474408],
       [-2.74909942],
       [ 1.05755407],
       [-0.13854586],
       [-0.27897733],
       [-0.8420619 ],
...
       [ 0.01559936],
       [ 0.01728286],
       [-3.41079652],
       [ 1.13730321],
       [-1.02930307],
       [ 0.36429333],
       [ 1.25762234],
       [-0.56930807],
       [ 1.0927444 ],
       [-0.63402883],
       [ 1.09464338],
       [ 1.00657795],
       [-0.33836177],
       [-1.72896035],
       [ 0.90683661],
       [ 1.50052911],
       [ 0.50546301],
       [ 1.07271413],
       [ 1.81420756],
       [ 1.6566195 ]])
Coordinates:
  * time     (time) object 1200-01-31 00:00:00 ... 1283-04-30 00:00:00
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 6 7 8 9 10 11 12 1 2 3 4
Dimensions without coordinates: index